# Web Scrapping lab

In this lab you will scrappe this [website](https://books.toscrape.com/) of books.

You have to create a Pandas DataFrame with all the books listed in the page. Each row of the DataFrame should contain information of each book. In particular, the DataFrmae must contain:

* category
* title
* price
* stock availability
* star rating (number of stars)
* description
* UPC

Happy scrapping!



# Server verification

Load the needed libraries, and make sure thar you can obtain the correct status code.

In [5]:
# Your code here
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

url = "https://books.toscrape.com/"
response = requests.get(url)
response

<Response [200]>

# Book categories

Create the code to collect the **relative urls** from the left panel to obtain a list with all the book categories.

In [9]:
# Your code here

In [11]:
soup = BeautifulSoup(response.content, "html.parser")

In [13]:
type(soup)

bs4.BeautifulSoup

In [15]:
categories = soup.find("ul", class_="nav nav-list").find("ul").find_all("a")
category_urls = ["https://books.toscrape.com/" + category["href"] for category in categories]

# Example output
for category in category_urls:
    print(category)

https://books.toscrape.com/catalogue/category/books/travel_2/index.html
https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
https://books.toscrape.com/catalogue/category/books/classics_6/index.html
https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html
https://books.toscrape.com/catalogue/category/books/romance_8/index.html
https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html
https://books.toscrape.com/catalogue/category/books/fiction_10/index.html
https://books.toscrape.com/catalogue/category/books/childrens_11/index.html
https://books.toscrape.com/catalogue/category/books/religion_12/index.html
https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html
https://books.toscrape.com/catalogue/category/books/music_14/index.html
https://books.

# Books in a given category

Use. web scrapping and list comprehension to obtain the **absolute** url of each book to be scraped

In [18]:
# Your code here

In [44]:
def get_book_urls(category_url):
    book_urls = []
    
    while True:
        response = requests.get(category_url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        books = soup.find_all("h3")
        for book in books:
            book_url = book.find("a")["href"]
            book_urls.append("https://books.toscrape.com/catalogue/" + book_url.replace('../../../', ''))
        
        next_page = soup.find("li", class_="next")
        if next_page:
            category_url = "https://books.toscrape.com/catalogue/" + next_page.find("a")["href"]
        else:
            break
    
    return book_urls

# Test on a single category
book_urls = get_book_urls(category_urls[0])
print(book_urls[:5])

['https://books.toscrape.com/catalogue/its-only-the-himalayas_981/index.html', 'https://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html', 'https://books.toscrape.com/catalogue/see-america-a-celebration-of-our-national-parks-treasured-sites_732/index.html', 'https://books.toscrape.com/catalogue/vagabonding-an-uncommon-guide-to-the-art-of-long-term-world-travel_552/index.html', 'https://books.toscrape.com/catalogue/under-the-tuscan-sun_504/index.html']


# Book details

Create a Python function that given a book_url as an input returns a dictionary with the following structure:

```Python
{"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc}
```

where `description` should contain the book's summary given in the Product description, and the values are the book's associated information.

In [ ]:
# Your code here

# Collect and store all the information from the books in a Pandas DataFrame

Start with the following dictionary:

```python
books_dict = {"Title": [], "Price": [], "Availability": [], "Rating": [], "Description": [], "UPC": [], "Category": [] }
```

Then, iterate over all the categories and all the books in a given category to collect any book information using the previous function. Fill the previous dictionary with the information about each book.

Show the first five rows of the previous final Pandas DataFrame.

Tip: You can use the function `tqdm` from the library `tqdm` to show a progress bar if in iterable of a for loop as shown below :wink: :

```python
from tqdm import tqdm

for elem in tqdm(iterable):
    # some code
```





In [83]:
from tqdm import tqdm

In [110]:
# Extract book details
title = soup.h1.text
price = soup.select_one('p.price_color').text
availability = soup.select_one('p.instock.availability').text.strip()
rating = soup.select_one('p.star-rating')['class'][1]
description = soup.select('meta[name="description"]')[0]['content'].strip()
upc = soup.select_one('table.table.table-striped tr:nth-child(1) td').text

# Display the extracted details
display(f"Title: {title}")
display(f"Price: {price}")
display(f"Availability: {availability}")
display(f"Rating: {rating}")
display(f"Description: {description[:10]}")
display(f"UPC: {upc}")

'Title: 1,000 Places to See Before You Die'

'Price: Â£26.08'

'Availability: In stock (1 available)'

'Rating: Five'

'Description: Around the'

'UPC: 228ba5e7577e1d49'

In [118]:
# Create a list to hold all book details
books_data = []

# Loop through each book URL
for book_url in book_urls:    
    title = soup.h1.text
    price = soup.select_one('p.price_color').text
    availability = soup.select_one('p.instock.availability').text.strip()
    rating = soup.select_one('p.star-rating')['class'][1]
    description = soup.select('meta[name="description"]')[0]['content'].strip()
    upc = soup.select_one('table.table.table-striped tr:nth-child(1) td').text
    
    # Add the book details to the list
    books_data.append({"Title": title, "Price": price, "Availability": availability, "Rating": rating, "Description": description, "UPC": upc})

# Convert the list into a Pandas df
books_df = pd.DataFrame(books_data)

# Display the head 5 rows of the df
display(books_df.head())

,Title,Price,Availability,Rating,Description,UPC
0,"1,000 Places to See Before You Die",Â£26.08,In stock (1 available),Five,"Around the World, continent by continent, here...",228ba5e7577e1d49
1,"1,000 Places to See Before You Die",Â£26.08,In stock (1 available),Five,"Around the World, continent by continent, here...",228ba5e7577e1d49
2,"1,000 Places to See Before You Die",Â£26.08,In stock (1 available),Five,"Around the World, continent by continent, here...",228ba5e7577e1d49
3,"1,000 Places to See Before You Die",Â£26.08,In stock (1 available),Five,"Around the World, continent by continent, here...",228ba5e7577e1d49
4,"1,000 Places to See Before You Die",Â£26.08,In stock (1 available),Five,"Around the World, continent by continent, here...",228ba5e7577e1d49
